# WEBSCRAPPING / VGSALES / METASCORE / MATACRITIC.COM

In [1]:
#pour effectuer correctement le scrapping nous aurons besoin de ces bibiliotéques : 

import re  #pour analyser les chaines de caractéres
import requests, bs4 #requests pour interagir avec le Web #bs4 pour BeautifulSoup
from requests import get #Pour atteindre envoyer une requete htpp 
from bs4 import BeautifulSoup #Pour scrapper le site avec les balises html
from IPython.core.display import clear_output  #Pour nettoyer le contenu de la trame (requete http)
from warnings import warn #controler les alertes
from time import sleep     #pour suspendre le temps d'execution durant un temps donné
from random import randint #pour renvoyer aléatoirement un entier
from time import time      #pour affiner le controle des requetes avec le temps (en secondes)



/var/folders/yc/4qdp1jds3xn35_s6g18nw2jc0000gn/T/ipykernel_9964/3957037850.py:7: DeprecationWarning: Importing clear_output from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import clear_output  #Pour nettoyer le contenu de la trame (requete http)


In [3]:
#pour scrapper le site il est important qu'il identifie via la requete HTTP que ne nous sommes pas un robot
#car cela peut bloquer l'acces au site via une requete http get

#on rentre les information de notre machine qui seront dans la trame http
headers = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko)Chrome/70.0.3538.77 Safari/537.36'}

#nous allons parcourir 196 pages, mais ici on initailise à bcp plus car on eu un pb avec l'iteration de la boucle
#et le nombre de pages donc on a décidé d'initialisé à plus en mettant un nombre assez large
pages = [str(i) for i in range(0,10000,1)]
#Listes des informations à prendre dans les pages nom,date de sortie, plateforme,metascore,userscore
names = []
release_dates = []
platform= []
meta_scores =[]
user_scores = []
#paramétrage de la boucle du monitoring afin de visualiser l'enchainement des requetes de la boucle
#plus de 196 pages et 56000 jeux à prendre
start_time = time()
requests = 0

#iteration de la boucle avec la liste page (0=>10000)
for page in pages:
    

    #avec la fonction get on créer la requete http
    game = get('https://www.metacritic.com/browse/games/score/metascore/all/all/filtered?page='+ page,headers = headers)
    #pause dans la boucle de 8 à 20 secondes
    sleep(randint(8,20))
    #initialisition de la variable requetes et du temps d'execution pour l'affichage du monitoring
    requests += 1
    elapsed_time = time() - start_time
    #affichage du monitoring
    print('Request:{}; Frequency: {} requests/s'.format(requests, requests/elapsed_time))
    clear_output(wait = True)
    #affihce un message d'alerte si au moins 200 requetes ne sont pas executer
    if game.status_code != 200:
        warn('Request: {}; Status code: {}'.format(requests,response.status_code))
    #arrete la boucle à partir de plus de 196 requetes
    if requests > 196:
        warn('Number of requests was greater than expected.')
        break
    #analyser le contenu de la réponse de la requete http dans la variable game_soup 
    game_soup = BeautifulSoup(game.text, 'html.parser')
    #trouve le tag majeur pour nos infos à recupérer
    container = game_soup.find_all('td', class_ = 'clamp-summary-wrap')
    #itération dans le majeur tag trouver
    for con in container:
        #scrape le nom du jeux
        name = con.find('h3').text
        names.append(name)
        #scrape la date de sortie
        release_date = con.select('div.clamp-details span')[2].text
        release_dates.append(release_date)
        #scrape la plateforme
        plateforme = con.select('div.clamp-details span')[1].text.strip()
        platform.append(plateforme)
        #scrape metascore
        meta_score= con.select('a.metascore_anchor div')[0].text
        meta_scores.append(meta_score)
        #scrape user score
        user_score = con.select('a.metascore_anchor div')[2].text
        user_scores.append(user_score)
        
#sotck dans le df movie_df toutes les infos        
import pandas as pd
movie_df = pd.DataFrame({'Movie_names': names,
'Release_dates': release_dates,
'Platform': platform,
'Meta_scores': meta_scores,
'User_scores': user_scores})
print(movie_df.info())
movie_df.head(1)

/var/folders/yc/4qdp1jds3xn35_s6g18nw2jc0000gn/T/ipykernel_56772/253273133.py:45: UserWarning: Number of requests was greater than expected.
  warn('Number of requests was greater than expected.')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19519 entries, 0 to 19518
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Movie_names    19519 non-null  object
 1   Release_dates  19519 non-null  object
 2   Platform       19519 non-null  object
 3   Meta_scores    19519 non-null  object
 4   User_scores    19519 non-null  object
dtypes: object(5)
memory usage: 762.6+ KB
None


,Movie_names,Release_dates,Platform,Meta_scores,User_scores
0,The Legend of Zelda: Ocarina of Time,"November 23, 1998",Nintendo 64,99,9.1


In [72]:
#affiche les premiéres lignes

movie_df

,Movie_names,Release_dates,Platform,Meta_scores,User_scores
0,The Legend of Zelda: Ocarina of Time,"November 23, 1998",Nintendo 64,99,9.1
1,Tony Hawk's Pro Skater 2,"September 20, 2000",PlayStation,98,7.4
2,Grand Theft Auto IV,"April 29, 2008",PlayStation 3,98,7.8
3,SoulCalibur,"September 8, 1999",Dreamcast,98,8.4
4,Grand Theft Auto IV,"April 29, 2008",Xbox 360,98,7.9
...,...,...,...,...,...
19514,Vroom in the Night Sky,"April 5, 2017",Switch,17,3.2
19515,Leisure Suit Larry: Box Office Bust,"May 5, 2009",PlayStation 3,17,1.9
19516,Yaris,"October 10, 2007",Xbox 360,17,4.4
19517,Ride to Hell: Retribution,"June 24, 2013",PC,16,1.4


In [73]:
#rename du nouveau dataframe pour hamrmoniser 
game_df = movie_df.rename(columns={"Movie_names": "Name", "Release_dates": "Year"})

#convertion en format date 
game_df['Year'] = pd.to_datetime(game_df['Year'])

In [75]:
#extraction de l'année avec la fonction year
game_df['year'] = game_df['Year'].dt.year


In [48]:
#extraction du df en un csv
game_df.to_csv('metagames.csv')

# Jointure entre l'ancien Dataframe VGSALES et le nouveau DF avec les scores METAGAMES

In [37]:
#telechargement du df vgsales.csv 
#telechargement du df metagames.csv

metagames = pd.read_csv("/Users/walidbenghalia//PROJET VGSALES DATASCIENTEST/metagames.csv")
vgsales = pd.read_csv("~/Desktop/vgsales.csv")
display(vgsales)
display(metagames)

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37
...,...,...,...,...,...,...,...,...,...,...,...
16593,16596,Woody Woodpecker in Crazy Castle 5,GBA,2002.0,Platform,Kemco,0.01,0.00,0.00,0.00,0.01
16594,16597,Men in Black II: Alien Escape,GC,2003.0,Shooter,Infogrames,0.01,0.00,0.00,0.00,0.01
16595,16598,SCORE International Baja 1000: The Official Game,PS2,2008.0,Racing,Activision,0.00,0.00,0.00,0.00,0.01
16596,16599,Know How 2,DS,2010.0,Puzzle,7G//AMES,0.00,0.01,0.00,0.00,0.01


,Unnamed: 0,Name,Year,Platform,Meta_scores,User_scores,year
0,0,The Legend of Zelda: Ocarina of Time,1998-11-23,Nintendo 64,99,9.1,1998
1,1,Tony Hawk's Pro Skater 2,2000-09-20,PlayStation,98,7.4,2000
2,2,Grand Theft Auto IV,2008-04-29,PlayStation 3,98,7.8,2008
3,3,SoulCalibur,1999-09-08,Dreamcast,98,8.4,1999
4,4,Grand Theft Auto IV,2008-04-29,Xbox 360,98,7.9,2008
...,...,...,...,...,...,...,...
19514,19514,Vroom in the Night Sky,2017-04-05,Switch,17,3.2,2017
19515,19515,Leisure Suit Larry: Box Office Bust,2009-05-05,PlayStation 3,17,1.9,2009
19516,19516,Yaris,2007-10-10,Xbox 360,17,4.4,2007
19517,19517,Ride to Hell: Retribution,2013-06-24,PC,16,1.4,2013


In [41]:
#comparaison des deux dataframe 

display(vgsales.head(1))
display(metagames.head(1))

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74


,Unnamed: 0,Name,Year,Platform,Meta_scores,User_scores,year
0,0,The Legend of Zelda: Ocarina of Time,1998-11-23,Nintendo 64,99,9.1,1998


In [43]:
#VGSALES : Traitement
#remise à niveau du format year pour supprimer le .0

#supprimer les NAN pour convertir en int 
#vgsales.dropna(inplace = True)


#transfomation de Year en Int
vgsales['year'] = vgsales['Year'].astype(int)
vgsales



,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,year
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74,2006
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,1985
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82,2008
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00,2009
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,1996
...,...,...,...,...,...,...,...,...,...,...,...,...
16593,16596,Woody Woodpecker in Crazy Castle 5,GBA,2002.0,Platform,Kemco,0.01,0.00,0.00,0.00,0.01,2002
16594,16597,Men in Black II: Alien Escape,GC,2003.0,Shooter,Infogrames,0.01,0.00,0.00,0.00,0.01,2003
16595,16598,SCORE International Baja 1000: The Official Game,PS2,2008.0,Racing,Activision,0.00,0.00,0.00,0.00,0.01,2008
16596,16599,Know How 2,DS,2010.0,Puzzle,7G//AMES,0.00,0.01,0.00,0.00,0.01,2010


In [44]:
#suppression de la colonne Year mauvaise
vgsales = vgsales.drop(['Year'], axis=1)

In [45]:
vgsales

,Rank,Name,Platform,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,year
0,1,Wii Sports,Wii,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74,2006
1,2,Super Mario Bros.,NES,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,1985
2,3,Mario Kart Wii,Wii,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82,2008
3,4,Wii Sports Resort,Wii,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00,2009
4,5,Pokemon Red/Pokemon Blue,GB,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,1996
...,...,...,...,...,...,...,...,...,...,...,...
16593,16596,Woody Woodpecker in Crazy Castle 5,GBA,Platform,Kemco,0.01,0.00,0.00,0.00,0.01,2002
16594,16597,Men in Black II: Alien Escape,GC,Shooter,Infogrames,0.01,0.00,0.00,0.00,0.01,2003
16595,16598,SCORE International Baja 1000: The Official Game,PS2,Racing,Activision,0.00,0.00,0.00,0.00,0.01,2008
16596,16599,Know How 2,DS,Puzzle,7G//AMES,0.00,0.01,0.00,0.00,0.01,2010


In [48]:
#metagames ; traitement

metagames

#suppression de Unnamed: 0 / Year

metagames.drop(['Unnamed: 0','Year'],axis=1,inplace=True)

In [54]:
display(metagames.head(1))
display(vgsales.head(1))

,Name,Platform,Meta_scores,User_scores,year
0,The Legend of Zelda: Ocarina of Time,N64,99,9.1,1998


,Rank,Name,Platform,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,year
0,1,Wii Sports,Wii,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74,2006


In [38]:
#stockage liste des valeurs unique Platform des deux dataFrame pour analyser les différences
list_vgsales_plt = vgsales['Platform'].unique()
display(list_vgsales_plt)

list_metagames_plt = metagames['Platform'].unique()
display(list_metagames_plt)

array(['Wii', 'NES', 'GB', 'DS', 'X360', 'PS3', 'PS2', 'SNES', 'GBA',
       '3DS', 'PS4', 'N64', 'PS', 'XB', 'PC', '2600', 'PSP', 'XOne', 'GC',
       'WiiU', 'GEN', 'DC', 'PSV', 'SAT', 'SCD', 'WS', 'NG', 'TG16',
       '3DO', 'GG', 'PCFX'], dtype=object)

array(['Nintendo 64', 'PlayStation', 'PlayStation 3', 'Dreamcast',
       'Xbox 360', 'Wii', 'Xbox One', 'PC', 'Switch', 'PlayStation 2',
       'PlayStation 4', 'GameCube', 'Xbox', 'Wii U', 'PlayStation 5',
       'Xbox Series X', 'Game Boy Advance', '3DS', 'DS',
       'PlayStation Vita', 'PSP', 'Stadia'], dtype=object)

In [50]:
#Changement des valeurs unique de platfrome de metagames avec celles de vgsales

#On remplace les valeurs de metagames platform par celle de vgsales platform
dictionnaire = {'Nintendo 64':'N64','PlayStation':'PS','PlayStation 3':'PS3','Dreamcast':'DC','Xbox 360':'X360',
               'Wii':'Wii','Xbox One':'XOne','PC':'PC','PlayStation 2':'PS2','PlayStation 4':'PS4',
               'GameCube':'GC','Xbox':'XB','Wii U':'WiiU','Game Boy Advance':'GBA','3DS':'3DS','DS':'DS',
               'PlayStation Vita':'PSV','PSP':'PSP','Stadia':'SAT'}

In [51]:
#remplacer les valeurs de metagames platform par le dictionnaire
metagames['Platform'].replace(dictionnaire,inplace = True)

In [55]:
#merge de metagames et vgsales
vgsales_scores = metagames.merge(vgsales,on=['Name','year','Platform'],how='inner')

In [56]:
#affiche les premiéres ligne du df merger vgsales_scores
vgsales_scores

,Name,Platform,Meta_scores,User_scores,year,Rank,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,The Legend of Zelda: Ocarina of Time,N64,99,9.1,1998,95,Action,Nintendo,4.10,1.89,1.45,0.16,7.60
1,Tony Hawk's Pro Skater 2,PS,98,7.4,2000,226,Sports,Activision,3.05,1.41,0.02,0.20,4.68
2,Grand Theft Auto IV,PS3,98,7.8,2008,57,Action,Take-Two Interactive,4.76,3.76,0.44,1.62,10.57
3,SoulCalibur,DC,98,8.4,1999,5359,Fighting,Namco Bandai Games,0.00,0.00,0.34,0.00,0.34
4,Grand Theft Auto IV,X360,98,7.9,2008,52,Action,Take-Two Interactive,6.76,3.10,0.14,1.03,11.02
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5927,Homie Rollerz,DS,23,3.0,2008,11936,Racing,Destineer,0.07,0.00,0.00,0.01,0.07
5928,Charlie's Angels,GC,23,4.3,2003,15524,Action,Ubisoft,0.01,0.00,0.00,0.00,0.02
5929,Fast & Furious: Showdown,X360,22,1.2,2013,13356,Action,Activision,0.00,0.04,0.00,0.00,0.05
5930,Deal or No Deal,DS,20,2.2,2007,1045,Misc,Mindscape,1.15,0.41,0.00,0.15,1.71


In [57]:
vgsales_scores.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5932 entries, 0 to 5931
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Name          5932 non-null   object 
 1   Platform      5932 non-null   object 
 2   Meta_scores   5932 non-null   int64  
 3   User_scores   5932 non-null   object 
 4   year          5932 non-null   int64  
 5   Rank          5932 non-null   int64  
 6   Genre         5932 non-null   object 
 7   Publisher     5932 non-null   object 
 8   NA_Sales      5932 non-null   float64
 9   EU_Sales      5932 non-null   float64
 10  JP_Sales      5932 non-null   float64
 11  Other_Sales   5932 non-null   float64
 12  Global_Sales  5932 non-null   float64
dtypes: float64(5), int64(3), object(5)
memory usage: 648.8+ KB


In [58]:
#suppression des nan
vgsales_scores.dropna(inplace=True)

In [59]:
#dimension du df vgsales_scores
vgsales_scores.shape

(5932, 13)